In [ ]:
# Important: Ensure taxonomy is imported before running this script

import csv
import logging
from db import DB

# Initialize your database connection (assuming DB is your ORM class)
db = DB()

# Path to your CSV file
csv_file = 'data/with_ofss_ids.csv'

def load_csv(file_path):
    """
    Loads the data from a CSV file.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - list: Parsed data from the CSV file as a list of dictionaries.
    """
    data = []
    with open(file_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)
    return data

def upsert_financial_data(db, csv_data):
    """
    Upserts the financial data into the database.

    Parameters:
    - db: The DB instance from your ORM.
    - csv_data (list): The parsed CSV data.
    """

    unmapped_concept_names = set()

    try:
        for row in csv_data:
            concept_name = row['tag']

            if concept_name.endswith("Text"):
                logging.debug(f"Skipping concept: {concept_name}")
                continue

            statement_type = row['statement_type'] if row['statement_type'] else None
            balance = row['balance'] if row['balance'] else None
            period_type = row['period_type'] if row['period_type'] else None

            ofss_id = row['ofss_id'] if row['ofss_id'] else None

            if balance is not None:
                # Upsert balance type
                balance_type_id = db.upsert_entity('us_gaap_balance_type', {'balance': balance}, ['balance'])
            else:
                balance_type_id = None
            
            if period_type is not None:
                # Upsert period type
                period_type_id = db.upsert_entity('us_gaap_period_type', {'period_type': period_type}, ['period_type'])
            else:
                period_type_id = None
            
            if statement_type is not None:
                # Upsert statement type
                statement_type_id = db.upsert_entity('us_gaap_statement_type', {'statement_type': statement_type}, ['statement_type'])
            else:
                statement_type_id = None
            
            # Upsert us_gaap_concept (the tag itself)
            concept_data = {
                'name': concept_name,
                'balance_type_id': balance_type_id,
                'period_type_id': period_type_id,
            }
            
            # Look up the tag ID
            concept_row = db.get(
                "SELECT id FROM us_gaap_concept WHERE name = %s",
                ["id"],
                params=(concept_data['name'],)
            )

            if concept_row.empty:
                logging.warning("Skipping unmapped concept: %s", concept_name)
                unmapped_concept_names.add(concept_name)
                continue

            concept_id = concept_row.iloc[0]['id']

            if ofss_id is not None:
                # Aassociate tag with the ofss category
                db.upsert_entity('us_gaap_concept_ofss_category', {
                    'us_gaap_concept_id': concept_id,
                    'ofss_category_id': ofss_id,
                    'is_manually_mapped': True # Hardcoded
                }, ['us_gaap_concept_id', 'ofss_category_id'])

                # Note: This is ideal, but must know all category ids first
                # Upsert associations for `us_gaap_concept_ofss_category`
                # db.upsert_and_cleanup(
                #     table_name="us_gaap_concept_ofss_category",
                #     entity_id_name="us_gaap_concept_id",
                #     parent_id_name="ofss_category_id",
                #     parent_id=concept_id,
                #     associate_entities=[{'ofss_category_id': ofss_category_id} for ofss_category_id in [ofss_id]],
                #     upsert_datetime_field=None,
                #     unique_fields=["us_gaap_concept_id", "ofss_category_id"]
                # )

                if statement_type_id is not None:
                    # Associate tag with statement type
                    db.upsert_entity('us_gaap_concept_statement_type', {
                        'us_gaap_concept_id': concept_id,
                        'us_gaap_statement_type_id': statement_type_id,
                        'is_manually_mapped': True # Hardcoded
                    }, ['us_gaap_concept_id', 'us_gaap_statement_type_id'])

                    # Note: This is ideal, but must know all statement ids first
                    # db.upsert_and_cleanup(
                    #     table_name="us_gaap_concept_statement_type",
                    #     entity_id_name="us_gaap_concept_id",
                    #     parent_id_name="us_gaap_statement_type_id",
                    #     parent_id=concept_id,
                    #     associate_entities=[{'us_gaap_statement_type_id': statement_type_id} for statement_type_id in [statement_type_id]],
                    #     upsert_datetime_field=None,
                    #     unique_fields=["us_gaap_concept_id", "us_gaap_statement_type_id"]
                    # )

            logging.debug(f"Upserted data for concept: {concept_name} with ofss_id: {ofss_id}")

        for concept_name in unmapped_concept_names:
            logging.warning(f"Unmapped concept: {concept_name}")
        logging.warning(f"Total unmapped concepts: {len(unmapped_concept_names)}")

        logging.info('Financial statement data has been successfully upserted.')
    except Exception as e:
        logging.error(f"Error upserting financial statement data: {e}")
        raise

if __name__ == "__main__":
    # Load the CSV data
    csv_data = load_csv(csv_file)

    # Upsert the data into the database
    upsert_financial_data(db, csv_data)


2025-04-10 09:31:45,933 - [2156352895.py:131] - DEBUG - Upserted data for concept: AccountsNotesAndLoansReceivableNetCurrent with ofss_id: 4
2025-04-10 09:31:45,945 - [2156352895.py:131] - DEBUG - Upserted data for concept: ReceivablesLongTermContractsOrPrograms with ofss_id: 22
2025-04-10 09:31:45,951 - [2156352895.py:131] - DEBUG - Upserted data for concept: AccountsReceivableGrossNoncurrent with ofss_id: 5
2025-04-10 09:31:45,957 - [2156352895.py:131] - DEBUG - Upserted data for concept: AccountsReceivableNet with ofss_id: 4
2025-04-10 09:31:45,962 - [2156352895.py:131] - DEBUG - Upserted data for concept: AccountsReceivableNetCurrent with ofss_id: 4
2025-04-10 09:31:45,968 - [2156352895.py:131] - DEBUG - Upserted data for concept: AccountsReceivableNetNoncurrent with ofss_id: 5
2025-04-10 09:31:45,974 - [2156352895.py:131] - DEBUG - Upserted data for concept: AccretionAmortizationOfDiscountsAndPremiumsInvestments with ofss_id: 115
2025-04-10 09:31:45,979 - [2156352895.py:131] - DEB

2025-04-10 09:31:46,040 - [2156352895.py:131] - DEBUG - Upserted data for concept: PaymentsToAcquireBusinessesAndInterestInAffiliates with ofss_id: 114
2025-04-10 09:31:46,046 - [2156352895.py:131] - DEBUG - Upserted data for concept: PaymentsToAcquireBusinessesNetOfCashAcquired with ofss_id: 114
2025-04-10 09:31:46,051 - [2156352895.py:131] - DEBUG - Upserted data for concept: PaymentsForProceedsFromBusinessesAndInterestInAffiliates with ofss_id: 117
2025-04-10 09:31:46,056 - [2156352895.py:131] - DEBUG - Upserted data for concept: OtherPaymentsToAcquireBusinesses with ofss_id: 114
2025-04-10 09:31:46,062 - [2156352895.py:131] - DEBUG - Upserted data for concept: PaymentsForProceedsFromRealEstatePartnershipInvestmentNet with ofss_id: 117
2025-04-10 09:31:46,067 - [2156352895.py:131] - DEBUG - Upserted data for concept: PaymentsToAcquirePartnersInterestInRealEstatePartnershipNetOfCashAcquired with ofss_id: 114
2025-04-10 09:31:46,071 - [2156352895.py:131] - DEBUG - Upserted data for co